<a href="https://colab.research.google.com/github/sajivhar4118/Rat-NOR/blob/main/Training_YoloV9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train and Run YOLOv9 on a Custom Dataset

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
!nvidia-smi

Mon Oct 14 16:15:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Clone and Install

**NOTE:** YOLOv9 is very new. At the moment, we recommend using a fork of the main repository. The `detect.py` script contains a bug that prevents inference. This bug is patched in the fork.

In [3]:
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

Cloning into 'yolov9'...
remote: Enumerating objects: 325, done.
remote: Total 325 (delta 0), reused 0 (delta 0), pack-reused 325 (from 1)
Receiving objects: 100% (325/325), 2.25 MiB | 4.84 MiB/s, done.
Resolving deltas: 100% (162/162), done.
/content/yolov9
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00


**NOTE:** Let's install the [`roboflow`](https://pypi.org/project/roboflow) package, which we will use to download our dataset from [Roboflow Universe](https://universe.roboflow.com/).

In [4]:
!pip install -q roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.5 MB/s eta 0:00:00


## Download model weights

**NOTE:** In the YOLOv9 paper, versions `yolov9-s` and `yolov9-m` are also mentioned, but the weights for these models are not yet available in the YOLOv9 [repository](https://github.com/WongKinYiu/yolov9).

In [7]:
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [6]:
!ls -la {HOME}/weights

total 402444
drwxr-xr-x 2 root root      4096 Oct 14 16:15 .
drwxr-xr-x 1 root root      4096 Oct 14 16:15 ..
-rw-r--r-- 1 root root  51508261 Feb 18  2024 gelan-c.pt
-rw-r--r-- 1 root root 117203713 Feb 18  2024 gelan-e.pt
-rw-r--r-- 1 root root 103153312 Feb 18  2024 yolov9-c.pt
-rw-r--r-- 1 root root 140217688 Feb 18  2024 yolov9-e.pt


## Authenticate and Download the Dataset

**NOTE:** The dataset must be saved inside the `{HOME}/yolov9` directory, otherwise, the training will not succeed.

In [6]:
%cd {HOME}/yolov9

/content/yolov9


**NOTE:** In this tutorial, I will use the [football-players-detection](https://universe.roboflow.com/roboflow-jvuqo/football-players-detection-3zvbc) dataset. Feel free to replace it with your dataset in YOLO format or use another dataset available on [Roboflow Universe](https://universe.roboflow.com). Additionally, if you plan to deploy your model to Roboflow after training, make sure you are the owner of the dataset and that no model is associated with the version of the dataset you are going to training on.

In [8]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="J7z40ETZmhYtsMn31GeD")
project = rf.workspace("ratnor").project("rat_nor")
version = project.version(6)
dataset = version.download("yolov9")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to RAT_NOR-6 in yolov9:: 100%|██████████| 206/206 [00:00<00:00, 6064.41it/s]


## Train Custom Model

In [ ]:
%cd {HOME}/yolov9

!python train.py \
--batch 16 --epochs 100 --img 480 --device 0 --min-items 0 --close-mosaic 15 \
--data /content/yolov9/RAT_NOR-6/data.yaml \
--weights {HOME}/weights/gelan-c.pt \
--cfg /content/yolov9/models/detect/gelan-c.yaml \
--hyp hyp.scratch-high.yaml

/content/yolov9
2024-10-01 00:02:06.001771: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 00:02:06.021962: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-01 00:02:06.028007: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-01 00:02:06.042408: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-01 00:02:07.288587: W

**NOTE:** By default, the results of each subsequent training sessions are saved in `{HOME}/yolov9/runs/train/`, in directories named `exp`, `exp2`, `exp3`, ... You can override this behavior by using the `--name` parameter.

In [ ]:
!ls {HOME}/yolov9/runs/train/

exp  exp2


## Validate Custom Model

In [8]:
%cd {HOME}/yolov9

!python val.py \
--img 640 --batch 32 --conf 0.001 --iou 0.7 --device 0 \
--data {dataset.location}/data.yaml \
--weights {HOME}/yolov9/runs/train/exp4/weights/best.pt

/content/yolov9
val: data=/content/yolov9/RAT_NOR-6/data.yaml, weights=['/content/yolov9/runs/train/exp4/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.7, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False, min_items=0
YOLOv5 🚀 1e33dbb Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

/content/yolov9/models/experimental.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the function

## Inference with Custom Model

In [ ]:
!python detect.py \
--img 480 --conf 0.5 --device 0 \
--weights /content/best.pt \
--source "/content/videos" \
--save-txt

detect: weights=['/content/best.pt'], source=/content/videos, data=data/coco128.yaml, imgsz=[480, 480], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 1e33dbb Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

/content/yolov9/models/experimental.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to

In [ ]:
import os
import csv
import re

# Define screen dimensions
SCREEN_WIDTH = 852
SCREEN_HEIGHT = 480

# Function to convert relative coordinates to pixels
def to_pixel_coords(relative_x, relative_y):
    pixel_x = round(relative_x * SCREEN_WIDTH)
    pixel_y = round(relative_y * SCREEN_HEIGHT)
    return pixel_x, pixel_y

# Directory containing the text files
labels_dir = "/content/yolov9/runs/detect/exp5/labels"

# Directory where the CSV files will be saved
output_dir = "/content/csv/"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Loop over the range of sets (1 to 18)
for i in range(1, 19):
    # Create the output CSV file for this set
    output_file = os.path.join(output_dir, f"video_frames_R{i}.csv")

    # Create a list to store (frame_number, file_path) tuples
    frame_files = []

    # Process each text file
    for filename in os.listdir(labels_dir):
        if filename.startswith(f"NOR10_Acq_R{i}") and filename.endswith(".txt"):
            # Extract frame number from the end of the filename (digits before .txt)
            match = re.search(r'_(\d+)\.txt$', filename)
            if match:
                frame_number = int(match.group(1))  # Extract the frame number as an integer
                file_path = os.path.join(labels_dir, filename)
                frame_files.append((frame_number, file_path))

    # Sort the frame files by frame number
    frame_files.sort(key=lambda x: x[0])

    # Write header to the CSV file
    with open(output_file, mode='w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["frame_number", "br_x", "br_y", "tl_x", "tl_y"])

        # Process sorted files
        for frame_number, file_path in frame_files:
            # Read the coordinates from the file
            with open(file_path, 'r') as f:
                lines = f.readlines()

                # Make sure there are enough lines to avoid index errors
                if len(lines) >= 2:
                    br_data = lines[0].split()
                    tl_data = lines[1].split()

                    # Extract relative coordinates
                    br_relative_x = float(br_data[1])
                    br_relative_y = float(br_data[2])
                    tl_relative_x = float(tl_data[1])
                    tl_relative_y = float(tl_data[2])

                    # Convert coordinates to pixels
                    br_x, br_y = to_pixel_coords(br_relative_x, br_relative_y)
                    tl_x, tl_y = to_pixel_coords(tl_relative_x, tl_relative_y)

                    # Write to CSV
                    csvwriter.writerow([frame_number, br_x, br_y, tl_x, tl_y])

    print(f"CSV file created: {output_file}")


CSV file created: /content/csv/video_frames_R1.csv
CSV file created: /content/csv/video_frames_R2.csv
CSV file created: /content/csv/video_frames_R3.csv
CSV file created: /content/csv/video_frames_R4.csv
CSV file created: /content/csv/video_frames_R5.csv
CSV file created: /content/csv/video_frames_R6.csv
CSV file created: /content/csv/video_frames_R7.csv
CSV file created: /content/csv/video_frames_R8.csv
CSV file created: /content/csv/video_frames_R9.csv
CSV file created: /content/csv/video_frames_R10.csv
CSV file created: /content/csv/video_frames_R11.csv
CSV file created: /content/csv/video_frames_R12.csv
CSV file created: /content/csv/video_frames_R13.csv
CSV file created: /content/csv/video_frames_R14.csv
CSV file created: /content/csv/video_frames_R15.csv
CSV file created: /content/csv/video_frames_R16.csv
CSV file created: /content/csv/video_frames_R17.csv
CSV file created: /content/csv/video_frames_R18.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
open(/content/yolov9/runs/detect/exp/labelsNOR10_Acq_R2_5272.txt)

SyntaxError: invalid syntax (<ipython-input-22-e84a6eb6a5b1>, line 1)